# Mergear shps para crear las capas de CartoDB

CartoDB permite el uso de sólo 4 capas para sus usuarios básicos, de manera que se mergearán los shps creando 4 capas principales: 
    1. Divisiones políticas de la CABA y sus indicadores (radios, fracciones, barrios y comunas)
    2. Points: estaciones de subte, metrobús y ferrocarril.
    3. Lines: líneas de subte, metrobús, ferrocarril y colectivos.
    4. Buffers: buffers de los points y sus indicadores.

In [89]:
import os
from scripts.path_finders import get_indicators_shp_path, get_indicators_dir, get_indicators_path
from scripts.cartodb_layers import merge_shapefiles
import shapefile
from scripts.create_indicators import get_or_create_indicators_df

In [90]:
paths = {
    "RADIO": get_indicators_shp_path("radios_censo_2010"),
    "BARRIO": get_indicators_shp_path("barrios_censo_2010"),
    "DPTO": get_indicators_shp_path("comunas_censo_2010"),
    "FRAC": get_indicators_shp_path("fracciones_censo_2010")
}

In [106]:
divs_path = os.path.join(get_indicators_dir(), "divisiones", "divisiones")

In [108]:
w = merge_shapefiles(paths, divs_path)

Merging FRAC /Users/abenassi/github/tod/indicadores/fracciones_censo_2010/fracciones_censo_2010
Merging DPTO /Users/abenassi/github/tod/indicadores/comunas_censo_2010/comunas_censo_2010
Merging RADIO /Users/abenassi/github/tod/indicadores/radios_censo_2010/radios_censo_2010
Merging BARRIO /Users/abenassi/github/tod/indicadores/barrios_censo_2010/barrios_censo_2010
